In [36]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2018, 1, 1).timestamp())  # Jan 1, 2010
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/mvrv_less_155',
    'https://api.glassnode.com/v1/metrics/indicators/sopr_less_155',
    'https://api.glassnode.com/v1/metrics/supply/sth_profit_loss_ratio'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

In [37]:
def calculate_sthunrealized_plratio_logpercentile(df, column='sth_profit_loss_ratio', perc_window=90, norm_window=90, normalization='maxmin'):
    # Apply log transformation to the input column
    log_values = np.log(df[column])
    
    # Calculate percentile
    percentile = log_values.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

In [38]:
def calculate_sthmvrvratio_logpercentile(df, column='mvrv_less_155', perc_window=90, norm_window=90, normalization='maxmin'):
    # Apply log transformation to the input column
    log_values = np.log(df[column])
    
    # Calculate percentile
    percentile = log_values.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

In [39]:
def calculate_sthsoprratio_smapercentile(df, column='sopr_less_155', perc_window=90, norm_window=90, normalization='maxmin'):
    # Apply 7-day Simple Moving Average (SMA)
    sma_values = df[column].rolling(window=7).mean()
    
    # Calculate percentile
    percentile = sma_values.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

In [40]:
# Apply the functions to the merged dataframe with maxmin normalization
merged_df['sthunrealized_plratio_logpercentile_maxmin'] = calculate_sthunrealized_plratio_logpercentile(merged_df, normalization='maxmin')
merged_df['sthmvrvratio_logpercentile_maxmin'] = calculate_sthmvrvratio_logpercentile(merged_df, normalization='maxmin')
merged_df['sthsoprratio_smapercentile_maxmin'] = calculate_sthsoprratio_smapercentile(merged_df, normalization='maxmin')

In [41]:
# ... (previous code remains the same)

# Visualization section

def create_indicator_chart(merged_df, indicator_column, chart_title):
    # Filter data for the last year
    one_year_ago = datetime.now() - timedelta(days=365)
    merged_df_last_year = merged_df[merged_df.index > one_year_ago]

    # Define a consistent grey color
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey

    # Create the visualization
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add price trace
    fig.add_trace(
        go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
        secondary_y=False,
    )

    # Add indicator trace
    indicator = merged_df_last_year[indicator_column]
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator,
            name=indicator_column,
            line=dict(color='green', width=2),
            fill='tozeroy',
            fillcolor='rgba(0,255,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add red color for negative values
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator < 0, 0),
            name=f"{indicator_column} (Negative)",
            line=dict(color='red', width=2),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add y=0 line on top (without adding to legend)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=[0] * len(merged_df_last_year),
            showlegend=False,
            line=dict(color=GREY_COLOR, width=2),
            hoverinfo='skip'
        ),
        secondary_y=True,
    )

    # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
    for month in [1, 3, 5, 7, 9, 11]:
        for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
            date = pd.Timestamp(year=year, month=month, day=1)
            if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
                fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

    # Get the last value of the indicator
    last_value = indicator.iloc[-1]
    last_date = indicator.index[-1]

    # Determine the color based on the last value
    indicator_color = 'green' if last_value >= 0 else 'red'

    # Add annotation for the last value
    fig.add_annotation(
        x=0.95,  # Place at the right edge of the chart
        y=last_value*0.88,  # Place at the vertical position of the last value
        xref="paper",
        yref="y2",  # Use the secondary y-axis for reference
        text=f"{last_value:.2f}",
        showarrow=False,
        font=dict(size=18, color=indicator_color),
        align="left",
        xanchor="left",
        yanchor="middle",
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        xaxis_title={
            'text': "Date",
            'font': {'color': 'black', 'size': 18}
        },
        showlegend=False,  # Remove legend
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,  # Set width for 16:9 aspect ratio
        height=450,  # Set height for 16:9 aspect ratio
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 14},
        zeroline=False
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=True, 
        range=[-1, 1], 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        side='right',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )

    return fig

# Create charts for each indicator
indicators = [
    ('sthunrealized_plratio_logpercentile_maxmin', "Bitcoin: STH Supply in P/L Ratio"),
    ('sthmvrvratio_logpercentile_maxmin', "Bitcoin: STH MVRV Ratio"),
    ('sthsoprratio_smapercentile_maxmin', "Bitcoin: STH SOPR Ratio")
]

for indicator, title in indicators:
    fig = create_indicator_chart(merged_df, indicator, title)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_last_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [42]:
merged_df.tail()

,price_usd_close,mvrv_less_155,sopr_less_155,sth_profit_loss_ratio,sthunrealized_plratio_logpercentile_maxmin,sthmvrvratio_logpercentile_maxmin,sthsoprratio_smapercentile_maxmin
t,,,,,,,
2024-09-23,63322.551391,1.017277,1.009015,1.535264,0.926829,0.926829,0.954023
2024-09-24,64338.712500,1.031250,1.008584,2.286069,1.000000,1.000000,0.954023
2024-09-25,63078.050802,1.010602,1.012260,1.121780,0.662651,0.783133,0.954023
2024-09-26,65134.209238,1.039131,1.013306,3.042726,1.000000,1.000000,0.862069
2024-09-27,65767.337050,1.049583,1.020292,3.338955,1.000000,1.000000,0.885057
